In [1]:
import numpy as np
import pandas as pd
import awkward as ak
import pyarrow as pa
import numba
import tpx3awkward as tpx

## Loading data in and clustering with old way

In [2]:
df = pd.read_parquet("/nsls2/data/chx/shared/tpx/shared_data/public_example_raw_events.parquet")
df = tpx.drop_zero_tot(df)
np_labels, np_events = tpx.cluster_df_optimized(df)
df['cluster_id'] = np_labels
print(len(df))
print(max(np_labels))
cluster_array = tpx.group_indices(np_labels)
data = tpx.centroid_clusters(cluster_array, np_events)
cdf = pd.DataFrame(tpx.ingest_cent_data(data)).sort_values("t").reset_index(drop=True)
cdf
print(len(cdf))

26235549
20914302
20914303


## Clustering data with awkward

In [3]:
@numba.jit(nopython=True)
def cluster_ak(events, radius = tpx.DEFAULT_CLUSTER_RADIUS, tw = tpx.DEFAULT_CLUSTER_TW):
    n = len(events)
    labels = np.full(n, -1, dtype=np.int64)
    cluster_id = 0

    max_time = radius * tw  # maximum time difference allowed for clustering
    radius_sq = radius ** 2

    for i in range(n):
        if labels[i] == -1:  # if event is unclustered
            labels[i] = cluster_id
            for j in range(i + 1, n):  # scan forward only
                if events[j].t - events[i].t > max_time:  # early exit based on time
                    break
                # Compute squared Euclidean distance
                dx = events[i].x - events[j].x
                dy = events[i].y - events[j].y
                dt = (events[i].t // tw) - (events[j].t // tw)
                distance_sq = dx**2 + dy**2 + dt**2

                if distance_sq <= radius_sq:
                    labels[j] = cluster_id
            cluster_id += 1

    return labels

table = pa.Table.from_pandas(df, preserve_index=False)
# Make copy of schema with non-nullable fields.
schema = pa.schema([field.with_nullable(False) for field in table.schema])
events = ak.from_arrow(table.cast(schema))
labels = cluster_ak(events)
sorted_indices = np.argsort(labels)
ak_clusters = ak.without_field(ak.unflatten(events[sorted_indices], ak.run_lengths(labels[sorted_indices])), "cluster_id")
print(len(ak_clusters))

20914303


## Comparing Clustering

In [4]:
def old_clustering(df):
    np_labels, np_events = tpx.cluster_df_optimized(df)
    df['cluster_id'] = np_labels

old_clustering(df)

In [5]:
table = pa.Table.from_pandas(df, preserve_index=False)
# Make copy of schema with non-nullable fields.
schema = pa.schema([field.with_nullable(False) for field in table.schema])
events = ak.from_arrow(table.cast(schema))

def new_clustering(events):
    labels = cluster_ak(events)
    sorted_indices = np.argsort(labels)
    ak_clusters = ak.without_field(ak.unflatten(events[sorted_indices], ak.run_lengths(labels[sorted_indices])), "cluster_id")

new_clustering(events)

In [6]:
%timeit old_clustering(df)

1.32 s ± 69.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit new_clustering(events)

1.37 s ± 47.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Numba/Numpy Centroiding

In [8]:
def centroid_numpy(clusters, events):
    #sorted_cluster_indices = np.argsort(clusters)
    cluster_array = tpx.group_indices(clusters)
    return tpx.centroid_clusters(cluster_array, events)
    
centroid_numpy(np_labels, np_events)

(array([  661703799,   661703802,   661703868, ..., 11746501989,
        11746502539, 11746502934], dtype=uint64),
 array([364.     , 145.28572, 180.     , ...,  28.     , 174.     ,
        219.     ], dtype=float32),
 array([ 45. , 109. , 206. , ..., 392. , 128. , 169.5], dtype=float32),
 array([275, 125, 375, ..., 375, 250, 200], dtype=uint32),
 array([275, 175, 375, ..., 375, 250, 400], dtype=uint32),
 array([1, 2, 1, ..., 1, 1, 2], dtype=uint8))

### Pandas Groupby Centroiding

In [9]:
sub_df = df[["x", "y", "t", "ToT", "cluster_id"]]

def centroid_pandas_groupby(df):
    df['x_weighted'] = df['x'] * df['ToT']
    df['y_weighted'] = df['y'] * df['ToT']
    
    grouped = df.groupby('cluster_id', sort=False)
    
    ToT_sum = grouped['ToT'].sum()
    ToT_max = grouped['ToT'].max()
    n = grouped.size()
    xc = grouped['x_weighted'].sum() / ToT_sum
    yc = grouped['y_weighted'].sum() / ToT_sum
    
    idxmax = grouped['ToT'].idxmax()
    t = df.loc[idxmax.values, 't'].values
    cluster_ids = idxmax.index.values
    
    return pd.DataFrame({
        't': t,
        'xc': xc.values,
        'yc': yc.values,
        'ToT_max': ToT_max.values,
        'ToT_sum': ToT_sum.values,
        'n': n.values
    }, index=cluster_ids)

centroid_pandas_groupby(sub_df)

/tmp/ipykernel_1368054/193542251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x_weighted'] = df['x'] * df['ToT']


,t,xc,yc,ToT_max,ToT_sum,n
0,661703799,364.000000,45.0,275,275,1
1,661703802,145.285714,109.0,125,175,2
2,661703868,180.000000,206.0,375,375,1
3,661704161,110.000000,215.0,400,400,1
4,661705668,141.000000,155.0,425,425,1
...,...,...,...,...,...,...
20914298,11746500740,268.000000,44.0,400,400,1
20914299,11746501908,182.000000,295.0,350,350,1
20914300,11746501989,28.000000,392.0,375,375,1
20914301,11746502539,174.000000,128.0,250,250,1


### Awkward+Numba Centroiding

In [10]:
@numba.jit(nopython=True)
def centroid_clusters_awkward_numba(
    awkward_array
):  

    num_clusters = len(awkward_array)
    
    t = np.zeros(num_clusters, dtype="uint64")
    xc = np.zeros(num_clusters, dtype="float32")
    yc = np.zeros(num_clusters, dtype="float32")
    ToT_max = np.zeros(num_clusters, dtype="uint32")
    ToT_sum = np.zeros(num_clusters, dtype="uint32")
    n = np.zeros(num_clusters, dtype="ubyte")

    cluster_id = 0
    for cluster in awkward_array:
        _ToT_sum = np.ushort(0)
        _ToT_max = np.ushort(0)
        for event in cluster:
            if event['ToT'] > _ToT_max:  # find the max ToT, assign, use that time
                _ToT_max = event['ToT']
                t[cluster_id] = event['t']
                ToT_max[cluster_id] = _ToT_max
            xc[cluster_id] += event['x'] * event['ToT']  # x and y centroids by time over threshold
            yc[cluster_id] += event['y'] * event['ToT']
            _ToT_sum += event['ToT']  # calcuate sum
            n[cluster_id] += np.ubyte(1)  # number of events in cluster

        xc[cluster_id] /= _ToT_sum  # normalize
        yc[cluster_id] /= _ToT_sum
        ToT_sum[cluster_id] = _ToT_sum
        cluster_id += 1

    return t, xc, yc, ToT_max, ToT_sum, n

In [11]:
%timeit awkward_clusters = centroid_clusters_awkward_numba(ak_clusters)

265 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit numpy_clusters = centroid_numpy(np_labels, np_events)

722 ms ± 62.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Awkward Only Centroiding

In [13]:
def centroid_awkward_without_events(a):
    n = ak.num(a, axis=1)
    ToT_max = ak.max(a['ToT'], axis=1)
    ToT_sum = ak.sum(a['ToT'], axis=1)
    xc = ak.sum(a['x'] * a['ToT'], axis=1) / ToT_sum
    yc = ak.sum(a['y'] * a['ToT'], axis=1) / ToT_sum

    t = ak.flatten(a['t'][ak.argmax(a['ToT'], axis=1, keepdims=True)])

    return ak.zip({
        "t": t,
        "xc": xc,
        "yc": yc,
        "ToT_max": ToT_max,
        "ToT_sum": ToT_sum,
        "n": n,
    }, depth_limit = 1)


def centroid_awkward_with_events(a):
    n = ak.num(a, axis=1)
    ToT_max = ak.max(a['ToT'], axis=1)
    ToT_sum = ak.sum(a['ToT'], axis=1)
    xc = ak.sum(a['x'] * a['ToT'], axis=1) / ToT_sum
    yc = ak.sum(a['y'] * a['ToT'], axis=1) / ToT_sum
    t = ak.flatten(a['t'][ak.argmax(a['ToT'], axis=1, keepdims=True)])


    return ak.zip({
        "t": t,
        "xc": xc,
        "yc": yc,
        "ToT_max": ToT_max,
        "ToT_sum": ToT_sum,
        "n": n,
        "events": a
    }, depth_limit = 1)


centroid_awkward_with_events(ak_clusters)
#centroid_ak_wo_events = centroid_awkward_without_events(ak_clusters)

<Array [{t: 661703799, xc: 364, ...}, ...] type='20914303 * {t: ?uint64, xc...'>

## Benchmark

In [14]:
import timeit
import statistics

def run_awkward_with_numba():
    centroid_clusters_awkward_numba(ak_clusters)

def run_numpy_with_numba():
    centroid_numpy(np_labels, np_events)

def run_pandas():
    centroid_pandas_groupby(sub_df)

def run_awkward_with_events():
    centroid_awkward_with_events(ak_clusters)

def run_awkward_without_events():
    centroid_awkward_without_events(ak_clusters)

# Benchmarking function
def benchmark(func, name, number=5):
    times = timeit.repeat(func, repeat=number, number=1)
    avg_time = statistics.mean(times)
    stddev = statistics.stdev(times)
    print(f"{name:40s} | Avg: {avg_time:.6f} s | Stddev: {stddev:.6f} s")

# Run benchmarks
benchmark(run_awkward_with_numba, "Awkward + Numba")
benchmark(run_numpy_with_numba, "NumPy + Numba")
benchmark(run_awkward_without_events, "Awkward Block Operations w/o Events")
benchmark(run_awkward_with_events, "Awkward Block Operations w/ Events")
benchmark(run_pandas, "Pandas groupby")

Awkward + Numba                          | Avg: 0.281359 s | Stddev: 0.009459 s
NumPy + Numba                            | Avg: 0.897429 s | Stddev: 0.001464 s
Awkward Block Operations w/o Events      | Avg: 2.418502 s | Stddev: 0.079508 s
Awkward Block Operations w/ Events       | Avg: 2.427737 s | Stddev: 0.105645 s
Pandas groupby                           | Avg: 4.793979 s | Stddev: 0.026456 s
